# Курсовая работа по дисциплине «Структуры и алгоритмы обработки данных»
## Выполнил студент группы БФИ2202 Лозицкий Александр

In [ ]:
# Установка необходимых модулей
%pip install tabulate matplotlib

In [ ]:
from typing import Callable
    
class Deque:
    def __init__(self, items: list = list()):
        self.items = items
    def pushToStart(self, item):
        self.items.insert(0, item)
    def pushToEnd(self, item):
        self.items.append(item)
    def popFromStart(self):
        return self.items.pop(0)
    def popFromEnd(self):
        return self.items.pop()
    def isEmpty(self):
        return len(self.items) == 0
    def __repr__(self) -> str: return str(self.items)
    
class HashMap:
    def __init__(self, length = 10):
        self.length = length
        self.data: list = [None for _ in range(length)]

    def getHash(self, key, rehash: bool = False, tries: int = 0) -> int:
        return (hash(key) + (rehash * tries)) % self.length

    def add(self, key, value):
        hash = self.getHash(key)
        tries = 0
        while self.data[hash] != None and self.data[hash][0] != key:
            tries += 1
            if tries == self.length:
                self.resize()
                self.add(key, value)
                return
            hash = self.getHash(key, True, tries)
        self.data[hash] = (key, value)
    
    def get(self, key):
        hash = self.getHash(key)
        tries = 0
        while self.data[hash] != None and self.data[hash][0] != key:
            tries += 1
            if tries == self.length:
                return None
            hash = self.getHash(key, True, tries)
        if self.data[hash] == None: return None
        return self.data[hash][1]
    
    def remove(self, key):
        hash = self.getHash(key)
        tries = 0
        while self.data[hash] != None and self.data[hash][0] != key:
            tries += 1
            if tries == self.length:
                return
            hash = self.getHash(key, True, tries)
        if self.data[hash] == None: return
        self.data[hash] = None

    def resize(self):
        old_data = self.data.copy()
        self.length *= 2
        self.data.clear()
        self.data.extend([None for _ in range(self.length)])
        for e in old_data:
            if e != None:
                self.add(*e)

    def __repr__(self) -> str: return str(self.data)

def sortFast(array, key: Callable = lambda x: x):
    def part(array, a, b):
        middle = (a+b)//2
        middle_el = key(array[middle])
        i = a - 1
        j = b + 1
        while i <= j:
            i += 1
            while key(array[i]) < middle_el: i += 1
            j -= 1
            while key(array[j]) > middle_el: j -= 1
            if i >= j: break
            tmp = array[j]
            array[j] = array[i]
            array[i] = tmp
        return j
    def _sortFast(array, a, b):
        if a < b:
            j = part(array, a, b)
            _sortFast(array, a, j)
            _sortFast(array, j+1, b)
    _sortFast(array, 0, len(array)-1)

In [ ]:
import csv
from datetime import datetime
from typing import NamedTuple

class DataRow(NamedTuple):
    order_id: str
    order_date: datetime
    product_name: str
    product_category: str
    product_sales_count: int
    product_price_per_unit: float
    price_total: float

def readData() -> list[DataRow]:
    data: list[DataRow] = []
    try:
        with open('data.csv', 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                data.append(DataRow(
                    order_id=row[0],
                    order_date=datetime.fromisoformat(row[1]),
                    product_name=row[2],
                    product_category=row[3],
                    product_sales_count=int(row[4]),
                    product_price_per_unit=float(row[5]),
                    price_total=float(row[6])
                ))
    except OSError:
        print("Ошибка открытия файла с данными")
    except csv.Error:
        print("Ошибка чтения данных")
    except:
        print("Ошибка")

    if len(data) == 0:
        print("Нет данных")
    return data
    
def processData(data: list[DataRow]) -> tuple[list[tuple], tuple, tuple, float]:
    total_revenue = 0
    data_map = HashMap()

    for row in data:
        row_current = data_map.get(row.product_name)
        if row_current == None:
            units_sold = 0
            money_earned = 0
        else: units_sold, money_earned = row_current
        data_map.add(row.product_name, (units_sold + row.product_sales_count, money_earned + row.price_total))

        total_revenue += row.price_total

    products = [x for x in data_map.data if x != None]

    sortFast(products, key=lambda x: x[1][1])
    products.reverse()

    max_revenue = products[0]
    max_sold = max(products, key=lambda x: x[1][0])

    return products, max_revenue, max_sold, total_revenue

In [ ]:
from tabulate import tabulate
import matplotlib.pyplot as plt
%matplotlib inline

def printReport(products: list[tuple], max_revenue: tuple, max_sold: tuple, total_revenue: float):
    report = f"""===== ОТЧЁТ =====
    Общая выручка: {total_revenue}

    Самый доходный товар: {max_revenue[0]}, {max_revenue[1][0]} шт. на сумму {max_revenue[1][1]}
    Самый часто продаваемый товар: {max_sold[0]}, {max_sold[1][0]} шт. на сумму {max_sold[1][1]}

    Информация по товарам:"""
    print(report)

    rows = Deque()
    rows.pushToEnd(["Название", "Количество проданных экземпляров", "Доход", "Доля от общего дохода"])
    for product in products:
        name, row_data = product
        row = (name, row_data[0], row_data[1], str(round(row_data[1]*100/total_revenue, 1)) + '%')
        rows.pushToEnd(row)
    print(tabulate(rows.items, headers='firstrow', tablefmt='fancy_grid'))
    rows.popFromStart()
    
    
    fig, ax = plt.subplots(figsize=(7, 0.3*len(rows.items)))

    x1 = range(len(rows.items), 0, -1)
    x2 = [x + 0.33 for x in x1]

    ax.set_ylabel('Товары')
    ax.set_xlabel('Доход', color = 'tab:blue')
    ax.tick_params('x', labelcolor = 'tab:blue')
    plt.barh(x1, [x[2] for x in rows.items], color = 'tab:blue', height=0.33, align='edge')

    ax2 = ax.twiny()
    ax2.set_xlabel('Продано шт.', color = 'tab:red')
    ax2.tick_params('x', labelcolor = 'tab:red')
    plt.barh(x2, [x[1] for x in rows.items], color = 'tab:red', height=0.33, align='edge')
    plt.yticks(x2, [x[0] for x in rows.items], wrap=True)

    ax.grid(axis='x', color='tab:blue', linestyle='--')
    ax2.grid(axis='x', color='tab:red', linestyle='--')

    plt.show()

In [ ]:
def main():
    data = readData()
    if len(data) == 0:
        print("Нет данных")
        return
    products, max_revenue, max_sold, total_revenue = processData(data)
    printReport(products, max_revenue, max_sold, total_revenue)

if __name__ == '__main__':
    main()